In [22]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# set executable path, then set up the URL (NASA Mars News (Links to an external site.)) for scraping
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
# assign the url & instruct the browser to Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)

# Optional delay for loading the page (a specific combination of tag (div) 
  # and attribute (list_text). example: ul.item_list would be found in HTML as <ul class="item_list">)
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [6]:
# set up the HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [7]:
# begin scraping: assign the title and summary text to variables 
  # we'll reference later
slide_elem.find('div', class_='content_title')

<div class="content_title">NASA's Perseverance Rover Bringing 3D-Printed Metal Parts to Mars</div>

In [8]:
# Use the parent element to find the first 'a' tag and save it as 'news_title'
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's Perseverance Rover Bringing 3D-Printed Metal Parts to Mars"

In [9]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"For hobbyists and makers, 3D printing expands creative possibilities; for specialized engineers, it's also key to next-generation spacecraft design."

### Featured Images.  
#(use markdown to separate the article scraping from the image scraping.  Select drop down in toolbar & select markdown.)

In [15]:
# set up the URL we are getting the images from
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [16]:
# we want to click the "Full Image" button. use the HTML tag in our code
# Find and click the full image button ([1] indicates we want to click the second 'button')
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [17]:
# need to click the More Info button to get to the next page.
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')


In [19]:
# Pull the most recent image each time the code is executed
# Find the relative image url (by inspecting the website code & using dev tools
    # we know it's in img class="fancybox-image" & src="image/featured/ . . .")
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars1.jpg'

In [21]:
# Use the base URL to create an absolute URL (add base URL to pull up image in browser)
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

MARKS FACTS TABLE

In [23]:
# scrape (grab) the entire table with Pandas' .read_html() function (at top import pandas)
  #create a new DataFrame from the HTML table. The Pandas function read_html() & [0] specifically 
  #searches for and returns the first list of tables found in the HTML
df = pd.read_html('https://galaxyfacts-mars.com')[0]
  # assign columns to the new DataFrame for additional clarity
df.columns=['description', 'Mars', 'Earth']
  # turning the Description column into the DataFrame's index. inplace=True means that the updated 
  # index will remain in place, without having to reassign the DataFrame to a new variable
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [24]:
# Use Pandas to easily convert our DataFrame back into HTML-ready code 
# using the .to_html() function for live web-site use
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [25]:
# end the automated browsing session
browser.quit()